In [ ]:
!mkdir -p ~/.kaggle # klasör oluşturduk
!cp kaggle.json ~/.kaggle/ # kaggle.json dosyasını bu dizine kopyalar
!chmod 600 ~/.kaggle/kaggle.json # dosyaya güvenlik izinlerini verdik

In [ ]:
# colab'da veri seti indirmek için
!kaggle datasets download -d manjilkarki/deepfake-and-real-images


In [ ]:
#Zip Dosyasının Açılması
!unzip deepfake-and-real-images.zip -d data

In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models

# Gerçek ve sahte görsel dosya yolları
real_image_path = 'data/Dataset/Train/Real'
fake_image_path = 'data/Dataset/Train/Fake'

def load_images_from_folder(folder, label):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.resize(img, (128, 128))  # Görüntü boyutunu 128x128 olarak ayarlıyoruz
            if len(img.shape) == 2:  # Eğer görüntü gri tonlamalıysa
                img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
            img = img / 255.0  # Görüntüleri normalize et
            images.append(img)
            labels.append(label)
    return np.array(images), np.array(labels)

# Gerçek ve sahte görselleri yükle
real_images, real_labels = load_images_from_folder(real_image_path, 1)  # Gerçekler için etiket 1
fake_images, fake_labels = load_images_from_folder(fake_image_path, 0)  # Sahte görseller için etiket 0

# Görselleri ve etiketleri birleştir
X = np.concatenate((real_images, fake_images), axis=0)
y = np.concatenate((real_labels, fake_labels), axis=0)

# Veriyi %80 eğitim, %20 test olarak ayıralım
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modeli tanımlama
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])


# Modeli derleme
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Modeli eğitme
history = model.fit(X_train, y_train, epochs=10, validation_split=0.2)

# Test veri seti üzerinde modelin doğruluğunu ölçme
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)

print(f"Test doğruluğu: {test_acc:.2f}")

In [ ]:
# modeli kaydetme

model.save('10_model.h5')

In [ ]:
# Modeli yükleme
loaded_model = tf.keras.models.load_model('model.h5')

real_image_path = 'data/Dataset/Validation/Real'
fake_image_path = 'data/Dataset/Validation/Fake'

real_images, real_labels = load_images_from_folder(real_image_path, 1)  # Gerçekler için etiket 1
fake_images, fake_labels = load_images_from_folder(fake_image_path, 0)  # Sahte görseller için etiket 0

# Görselleri ve etiketleri birleştir
X = np.concatenate((real_images, fake_images), axis=0)
y = np.concatenate((real_labels, fake_labels), axis=0)

# Test seti üzerinde değerlendirme
test_loss, test_acc = loaded_model.evaluate(X, y, verbose=2)
print(f"Yüklenen modelin test doğruluğu: {test_acc:.2f}")

# Tahmin yapma
predictions = loaded_model.predict(X)
predicted_classes = (predictions > 0.5).astype(int)

model colab linki: https://colab.research.google.com/drive/1pAZErQjCQvoVGIVMeuLscmaJRWaSSa6E#scrollTo=d7dcQcjhM3ef